# Planet — Corporate Hybrid Capacity Forecast (v6.x)  
## Confidence Intervals, Department Accuracy & Automatic Staffing Safety Factors

**Generated:** 2026-02-13 (Europe/Madrid)  
**Audience:** Workforce Management, Operations Leadership, Continuous Improvement, Data/Analytics  
**Purpose:** Produce **forecast + confidence intervals** and convert to **staffing plan (FTE)** with an **automatic safety factor** based on **department-level model accuracy**.

---
What this notebook adds (vs. the baseline hybrid pipeline)
- **P05/P95 prediction intervals** for monthly forecasts (when supported).
- **Backtesting-based accuracy** by **department + language** (sMAPE, MAE, Bias, Accuracy%, PI coverage).
- **Automatic FTE inflation/deflation** using a **policy-driven safety factor**:
  - Example: if Accuracy < 80%, apply +X% buffer on FTE.
  - If Accuracy is high and intervals are tight, optionally deflate slightly.

> **Disclaimer**: This is a planning support tool. Final staffing decisions should consider operational constraints (training, shrinkage, attrition, SLAs, launches, system incidents).


### 1) Setup

Expected inputs (same as baseline)
- `Incoming_new.xlsx` (Sheet `Main`)  
- `department.xlsx` (Sheet `map`)  
- `productivity_agents.xlsx`

Output
- `capacity_forecast_hybrid.xlsx` with:
  - `forecast_monthly` (forecast + P05/P95)
  - `accuracy_dept_monthly` (backtesting KPIs)
  - `daily_capacity_plan` (staffing plan) with:
    - `Safety_Factor_%`
    - `FTE_Adj`
    - `Buffer_FTE`

## 0. Config

In [13]:
import os
from pathlib import Path
import tkinter as tk
from tkinter import filedialog

# ------------------------------
# Interactive BASE_DIR selection
# ------------------------------
root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)  # brings dialog to front (Windows-friendly)

BASE_DIR = filedialog.askdirectory(title="Select BASE_DIR (CAPACITY folder)")
if not BASE_DIR:
    raise SystemExit("No folder selected. Execution stopped.")

BASE_DIR = str(Path(BASE_DIR).expanduser().resolve())

INPUT_DIR  = str(Path(BASE_DIR) / "input_model")
OUTPUT_DIR = str(Path(BASE_DIR) / "outputs")
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

INCOMING_SOURCE_PATH = str(Path(INPUT_DIR) / "Incoming_new.xlsx")  # Sheet 'Main'
INCOMING_SHEET = "Main"

DEPT_MAP_PATH = str(Path(INPUT_DIR) / "department.xlsx")
DEPT_MAP_SHEET = "map"

PRODUCTIVITY_PATH = str(Path(INPUT_DIR) / "productivity_agents.xlsx")

OUTPUT_XLSX = str(Path(OUTPUT_DIR) / "capacity_forecast_hybrid.xlsx")

# Validate expected structure/files
required_files = [Path(INCOMING_SOURCE_PATH), Path(DEPT_MAP_PATH), Path(PRODUCTIVITY_PATH)]
missing = [str(p) for p in required_files if not p.exists()]
if missing:
    raise FileNotFoundError(
        "Missing required input files:\n- " + "\n- ".join(missing) +
        "\n\nSelected BASE_DIR:\n" + BASE_DIR +
        "\n\nExpected structure:\n"
        "BASE_DIR/\n  input_model/\n    Incoming_new.xlsx\n    department.xlsx\n    productivity_agents.xlsx\n  outputs/\n"
    )

# ------------------------------
# Forecast configuration
# ------------------------------
H_MONTHS = 12
DAILY_HORIZON_DAYS = 90

ENABLE_MONTHLY_PI = True
PI_ALPHA = 0.05  # 95% PI -> P05/P95

ENABLE_DEPT_ACCURACY_TABLE = True
ACCURACY_BACKTEST_MONTHS = 6
ACCURACY_MIN_TRAIN_MONTHS = 12
ACCURACY_HORIZON_MONTHS = 1
ACCURACY_MAX_SPLITS = 6

SAFETY_POLICY = {
    "low_accuracy_threshold": 80.0,
    "low_accuracy_buffer_pct": 12.0,
    "mid_accuracy_threshold": 88.0,
    "mid_accuracy_buffer_pct": 6.0,
    "high_accuracy_threshold": 95.0,
    "high_accuracy_deflate_pct": -2.0,
    "max_pi_width_ratio_for_deflate": 0.35,
}

LANGUAGE_STRATEGY = "from_column"  # "from_column" or "fixed_shares"
LANGUAGE_SHARES = {
    "English": 0.6435, "French": 0.0741, "German": 0.0860,
    "Italian": 0.0667, "Portuguese": 0.0162, "Spanish": 0.1135
}

WEEKLY_FREQ = "W-WED"

# ------------------------------
# Imports
# ------------------------------
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from datetime import date, timedelta
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import STL

try:
    from prophet import Prophet
except Exception:
    try:
        from fbprophet import Prophet
    except Exception:
        Prophet = None

try:
    from tbats import TBATS
except Exception:
    TBATS = None

print("✅ Setup OK")
print("BASE_DIR:", BASE_DIR)
print("INPUT_DIR:", INPUT_DIR)
print("OUTPUT_DIR:", OUTPUT_DIR)
print("Prophet:", bool(Prophet), "| TBATS:", bool(TBATS))


✅ Setup OK
BASE_DIR: C:\Projects\Capacity_forecast_2026
INPUT_DIR: C:\Projects\Capacity_forecast_2026\input_model
OUTPUT_DIR: C:\Projects\Capacity_forecast_2026\outputs
Prophet: True | TBATS: False


### 1) Core helpers (metrics + transforms + prediction intervals)

- sMAPE and accuracy metrics  
- Safe log transforms  
- Interval (P05/P95) assembly in log-space  
- Simple daily outlier clipping


In [14]:
def smape(y_true, y_pred) -> float:
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    denom = (np.abs(y_true) + np.abs(y_pred))
    denom[denom == 0] = 1.0
    return float(np.mean(2.0 * np.abs(y_pred - y_true) / denom) * 100.0)

def _z_from_alpha(alpha: float) -> float:
    if abs(alpha - 0.10) < 1e-9: return 1.6448536269514722
    if abs(alpha - 0.05) < 1e-9: return 1.959963984540054
    if abs(alpha - 0.01) < 1e-9: return 2.5758293035489004
    return 1.959963984540054

def clamp_nonneg(a: np.ndarray) -> np.ndarray:
    a = np.array(a, dtype=float)
    a[~np.isfinite(a)] = 0.0
    return np.clip(a, 0.0, None)

def expm1_safe(x, cap_original: Optional[float] = None):
    a = np.array(x, dtype=float)
    a[~np.isfinite(a)] = -50.0
    a = np.maximum(a, -50.0)
    if cap_original and np.isfinite(cap_original) and cap_original > 0:
        log_cap = np.log1p(cap_original)
        a = np.minimum(a, log_cap)
    y = np.expm1(a)
    if cap_original and np.isfinite(cap_original) and cap_original > 0:
        y = np.minimum(y, cap_original)
    return np.clip(y, 0, None)

def compute_dynamic_cap(ts_m: pd.Series) -> float:
    if ts_m.empty or (ts_m.max() <= 0):
        return np.inf
    m12 = float(ts_m.tail(12).mean()) if len(ts_m) >= 3 else float(ts_m.mean())
    med, mx = float(ts_m.median()), float(ts_m.max())
    base = max(1.0, m12, med, 1.1 * mx)
    return base * 6.0

def compute_pi_from_log_mean_std(mean_log: np.ndarray,
                                 std_log: np.ndarray,
                                 cap: float,
                                 alpha: float = 0.05) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    z = _z_from_alpha(alpha)
    std_log = np.clip(np.array(std_log, dtype=float), 1e-8, None)
    lo_log = mean_log - z * std_log
    hi_log = mean_log + z * std_log
    mean = expm1_safe(mean_log, cap_original=cap)
    lo   = expm1_safe(lo_log,   cap_original=cap)
    hi   = expm1_safe(hi_log,   cap_original=cap)
    mean = clamp_nonneg(mean); lo = clamp_nonneg(lo); hi = clamp_nonneg(hi)
    lo2 = np.minimum(lo, hi); hi2 = np.maximum(lo, hi)
    return mean, lo2, hi2

def coverage_95(actual: np.ndarray, lo: np.ndarray, hi: np.ndarray) -> float:
    a = np.array(actual, dtype=float)
    l = np.array(lo, dtype=float)
    h = np.array(hi, dtype=float)
    m = np.isfinite(a) & np.isfinite(l) & np.isfinite(h)
    if m.sum() == 0:
        return np.nan
    ok = (a[m] >= l[m]) & (a[m] <= h[m])
    return float(ok.mean() * 100.0)

def clean_outliers_daily(g: pd.DataFrame, method="IQR") -> pd.DataFrame:
    g = g.copy()
    s = g.sort_values('Date')['ticket_total'].astype(float)
    ql = s.quantile(0.01)
    qh = s.quantile(0.99)
    g['ticket_total'] = s.clip(ql, qh).values
    return g


## 4) Loaders (Incoming, Department Map, Productivity)

Standardized loaders for Planet’s operational datasets.


In [15]:
def load_incoming(path: str, sheet_name: str) -> pd.DataFrame:
    df = pd.read_excel(path, sheet_name=sheet_name, engine="openpyxl")
    required = {"Date", "department_id"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Incoming missing columns: {sorted(list(missing))}. Found: {list(df.columns)}")

    if "ticket_total" not in df.columns:
        if "total_incoming" in df.columns:
            df["ticket_total"] = pd.to_numeric(df["total_incoming"], errors="coerce").fillna(0)
        elif {"incoming_from_customers", "incoming_from_transfers"}.issubset(df.columns):
            df["ticket_total"] = (
                pd.to_numeric(df["incoming_from_customers"], errors="coerce").fillna(0)
                + pd.to_numeric(df["incoming_from_transfers"], errors="coerce").fillna(0)
            )
        else:
            raise ValueError("Incoming must contain ticket_total or a known alternative column set.")

    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    if df["Date"].isna().any():
        bad = df.loc[df["Date"].isna()].head(5)
        raise ValueError(f"Some Date values could not be parsed. Example rows:\n{bad}")

    df["department_id"] = df["department_id"].astype(str).str.strip()
    df["ticket_total"] = pd.to_numeric(df["ticket_total"], errors="coerce").fillna(0.0).astype(float)

    if "department_name" not in df.columns:
        df["department_name"] = None
    if "vertical" not in df.columns:
        df["vertical"] = None
    if "language" not in df.columns:
        df["language"] = None

    if (LANGUAGE_STRATEGY == "from_column") and df["language"].notna().any():
        df["language"] = (df["language"].astype(str).str.strip().replace({"nan": None, "None": None}).fillna("English"))
    else:
        parts = []
        base = df.copy()
        for lang, w in LANGUAGE_SHARES.items():
            tmp = base.copy()
            tmp["language"] = lang
            tmp["ticket_total"] = tmp["ticket_total"] * float(w)
            parts.append(tmp)
        df = pd.concat(parts, ignore_index=True)

    return df

def load_dept_map(path: str, sheet: str) -> pd.DataFrame:
    mp = pd.read_excel(path, sheet_name=sheet, engine="openpyxl")
    rename_map = {
        'dept_id': 'department_id', 'dept_name': 'department_name', 'name': 'department_name',
        'segment': 'vertical', 'vertical_name': 'vertical'
    }
    mp = mp.rename(columns={k: v for k, v in rename_map.items() if k in mp.columns})
    if 'department_id' not in mp.columns:
        raise ValueError(f"Department map must contain department_id. Found: {list(mp.columns)}")
    mp['department_id'] = mp['department_id'].astype(str).str.strip()
    if 'department_name' in mp.columns:
        mp['department_name'] = mp['department_name'].astype(str).str.strip()
    if 'vertical' in mp.columns:
        mp['vertical'] = mp['vertical'].astype(str).str.strip()
    cols = [c for c in ['department_id','department_name','vertical'] if c in mp.columns]
    return mp[cols].drop_duplicates('department_id')

def apply_mapping(incoming: pd.DataFrame, mapping: pd.DataFrame) -> pd.DataFrame:
    merged = incoming.merge(mapping, on='department_id', how='left', suffixes=('', '_map'))
    if 'department_name_map' in merged.columns:
        merged['department_name'] = merged['department_name'].fillna(merged['department_name_map'])
    merged['department_name'] = merged['department_name'].fillna("Unknown")
    if 'vertical_map' in merged.columns:
        merged['vertical'] = merged['vertical'].fillna(merged['vertical_map'])
    merged['vertical'] = merged['vertical'].fillna("Unmapped")
    merged.drop(columns=[c for c in merged.columns if c.endswith('_map')], inplace=True, errors='ignore')
    return merged

def load_productivity(path: str) -> pd.DataFrame:
    df = pd.read_excel(path, engine="openpyxl")
    req = {'Date', 'agent_id', 'department_id', 'prod_total_model'}
    missing = req - set(df.columns)
    if missing:
        raise ValueError(f"productivity missing columns: {sorted(list(missing))}. Found: {list(df.columns)}")
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['department_id'] = df['department_id'].astype(str).str.strip()
    df['prod_total_model'] = pd.to_numeric(df['prod_total_model'], errors='coerce')
    prod_dept = (
        df.groupby('department_id', as_index=False)['prod_total_model']
        .mean().rename(columns={'prod_total_model': 'avg_tickets_per_agent_day'})
    )
    return prod_dept


## 5) Exogenous calendar (EU core)

Minimal holiday/event generator aligned to Planet’s operational footprint.


In [16]:
def _easter_sunday(year: int) -> date:
    a = year % 19
    b = year // 100
    c = year % 100
    d = b // 4
    e = b % 4
    f = (b + 8) // 25
    g = (b - f + 1) // 3
    h = (19 * a + b - d - g + 15) % 30
    i = c // 4
    k = c % 4
    L = (32 + 2 * e + 2 * i - h - k) % 7
    m = (a + 11 * h + 22 * L) // 451
    month = (h + L - 7 * m + 114) // 31
    day = ((h + L - 7 * m + 114) % 31) + 1
    return date(year, month, day)

def _last_friday_of_november(year: int) -> date:
    d = date(year, 11, 30)
    while d.weekday() != 4:
        d = d.replace(day=d.day - 1)
    return d

def build_eu_core_holidays(start_date: pd.Timestamp, end_date: pd.Timestamp) -> pd.DataFrame:
    years = list(range(start_date.year, end_date.year + 1))
    rows = []
    for y in years:
        rows += [
            {"ds": date(y,1,1), "type":"holiday", "weight":1.0},
            {"ds": date(y,5,1), "type":"holiday", "weight":1.0},
            {"ds": date(y,12,25), "type":"holiday", "weight":1.0},
            {"ds": date(y,12,26), "type":"holiday", "weight":1.0},
        ]
        easter = _easter_sunday(y)
        rows += [
            {"ds": easter - timedelta(days=2), "type":"holiday", "weight":1.0},
            {"ds": easter + timedelta(days=1), "type":"holiday", "weight":1.0},
        ]
        bf = _last_friday_of_november(y)
        rows += [
            {"ds": bf, "type":"event", "weight":0.6},
            {"ds": bf + timedelta(days=4), "type":"event", "weight":0.6},
        ]
    exo = pd.DataFrame(rows)
    exo["ds"] = pd.to_datetime(exo["ds"])
    exo = exo[(exo["ds"] >= start_date) & (exo["ds"] <= end_date)].reset_index(drop=True)
    return exo

def build_exogenous_calendar(incoming: pd.DataFrame, horizon_days: int):
    start = incoming['Date'].min() - pd.Timedelta(days=365)
    end = incoming['Date'].max() + pd.Timedelta(days=horizon_days)

    exo = build_eu_core_holidays(start, end)
    cal = pd.DataFrame({'ds': pd.date_range(start, end, freq='D')})
    exo_daily = cal.merge(exo, on='ds', how='left')
    exo_daily['is_holiday'] = (exo_daily['type'] == 'holiday').astype(int)
    exo_daily['is_event'] = (exo_daily['type'] == 'event').astype(int)
    exo_daily['weight_h'] = np.where(exo_daily['is_holiday'] == 1, exo_daily['weight'].fillna(1.0), 0.0)
    exo_daily['weight_e'] = np.where(exo_daily['is_event'] == 1, exo_daily['weight'].fillna(1.0), 0.0)
    exo_daily = exo_daily[['ds','is_holiday','is_event','weight_h','weight_e']]

    exo_daily['month'] = exo_daily['ds'].dt.to_period('M')
    exo_monthly = (exo_daily.groupby('month', as_index=False)
                   .agg(hol_count=('is_holiday','sum'),
                        evt_count=('is_event','sum'),
                        hol_weight_sum=('weight_h','sum'),
                        evt_weight_sum=('weight_e','sum')))
    return exo_daily, exo_monthly

def prepare_monthly_exog(exo_monthly: pd.DataFrame, ts_index: pd.PeriodIndex) -> pd.DataFrame:
    ex = exo_monthly.copy()
    if not pd.api.types.is_period_dtype(ex['month']):
        ex['month'] = pd.PeriodIndex(ex['month'], freq='M')
    ex = ex.set_index('month').reindex(ts_index).fillna(0.0)
    return ex[['hol_count','evt_count','hol_weight_sum','evt_weight_sum']]


## 6) Monthly models with prediction intervals (P05/P95)

We fit in **log1p space** and bring results back to original scale with safety capping.


In [17]:
def fit_prophet_monthly_log_with_pi(ts_m: pd.Series, exo_m: Optional[pd.DataFrame] = None):
    if Prophet is None or len(ts_m) < 6:
        return None, None, None

    cap = compute_dynamic_cap(ts_m)
    y = np.log1p(ts_m.values)
    dfp = pd.DataFrame({'ds': ts_m.index.to_timestamp(), 'y': y})
    if exo_m is not None:
        dfp = dfp.join(exo_m, on=ts_m.index).reset_index(drop=True)

    m = Prophet(weekly_seasonality=False, yearly_seasonality=True, daily_seasonality=False,
                interval_width=(1.0 - PI_ALPHA))
    if exo_m is not None:
        for col in exo_m.columns:
            m.add_regressor(col)
    m.fit(dfp)

    def _future(h_months: int):
        future = m.make_future_dataframe(periods=h_months, freq='MS')
        if exo_m is not None:
            idx_future = pd.period_range(ts_m.index[-1] + 1, periods=h_months, freq='M')
            pad = exo_m.iloc[[-1]].repeat(h_months)
            pad.index = idx_future
            ex_full = pd.concat([exo_m, pad], axis=0)
            for col in exo_m.columns:
                future[col] = ex_full[col].values
        return future

    def f_mean(h_months: int):
        pred = m.predict(_future(h_months))
        pred.index = pd.PeriodIndex(pred['ds'], freq='M')
        mu_log = pred['yhat'].iloc[-h_months:].values
        mean = expm1_safe(mu_log, cap_original=cap)
        return pd.Series(clamp_nonneg(mean), index=pred.index[-h_months:])

    def f_pi(h_months: int):
        pred = m.predict(_future(h_months))
        pred.index = pd.PeriodIndex(pred['ds'], freq='M')
        idx = pred.index[-h_months:]
        mu_log = pred['yhat'].iloc[-h_months:].values
        lo_log = pred['yhat_lower'].iloc[-h_months:].values
        hi_log = pred['yhat_upper'].iloc[-h_months:].values
        mean = clamp_nonneg(expm1_safe(mu_log, cap_original=cap))
        lo = clamp_nonneg(expm1_safe(lo_log, cap_original=cap))
        hi = clamp_nonneg(expm1_safe(hi_log, cap_original=cap))
        lo2 = np.minimum(lo, hi); hi2 = np.maximum(lo, hi)
        return (pd.Series(mean, index=idx), pd.Series(lo2, index=idx), pd.Series(hi2, index=idx))

    return m, f_mean, f_pi

def fit_arima_monthly_log_with_pi(ts_m: pd.Series, exo_m: Optional[pd.DataFrame] = None):
    cap = compute_dynamic_cap(ts_m)
    y = np.log1p(ts_m)

    best_aic, best_model, best_exog = np.inf, None, None
    pqs = [0, 1]
    seasonal = len(ts_m) >= 12
    PsQs = [0] if seasonal else [0]

    for p in pqs:
        for d in ([1] if len(ts_m) < 24 else [0, 1]):
            for q in pqs:
                for P in PsQs:
                    for D in ([0, 1] if seasonal else [0]):
                        for Q in PsQs:
                            try:
                                model = SARIMAX(
                                    y, order=(p, d, q),
                                    seasonal_order=(P, D, Q, 12 if seasonal else 0),
                                    exog=exo_m.values if exo_m is not None else None,
                                    enforce_stationarity=False, enforce_invertibility=False
                                ).fit(disp=False)
                                if model.aic < best_aic:
                                    best_aic = model.aic
                                    best_model = model
                                    best_exog = exo_m
                            except Exception:
                                continue

    def _future_exog(h_months: int, idx: pd.PeriodIndex):
        if best_exog is None:
            return None
        pad = best_exog.iloc[[-1]].repeat(h_months)
        pad.index = idx
        return pad.values

    def f_mean(h_months: int):
        idx = pd.period_range(ts_m.index[-1] + 1, periods=h_months, freq='M')
        if best_model is None:
            mu = np.full(h_months, float(np.nanmean(y.values)))
            mean = expm1_safe(mu, cap_original=cap)
            return pd.Series(clamp_nonneg(mean), index=idx)
        exf = _future_exog(h_months, idx)
        fc = best_model.get_forecast(h_months, exog=exf)
        mu_log = fc.predicted_mean.values
        mean = expm1_safe(mu_log, cap_original=cap)
        return pd.Series(clamp_nonneg(mean), index=idx)

    def f_pi(h_months: int):
        idx = pd.period_range(ts_m.index[-1] + 1, periods=h_months, freq='M')
        if best_model is None:
            mu_log = np.full(h_months, float(np.nanmean(y.values)))
            std_log = np.full(h_months, float(np.nanstd(y.values) if np.isfinite(np.nanstd(y.values)) else 0.5))
            mean, lo, hi = compute_pi_from_log_mean_std(mu_log, std_log, cap=cap, alpha=PI_ALPHA)
            return (pd.Series(mean, index=idx), pd.Series(lo, index=idx), pd.Series(hi, index=idx))
        exf = _future_exog(h_months, idx)
        fc = best_model.get_forecast(h_months, exog=exf)
        mu_log = fc.predicted_mean.values
        ci = fc.conf_int(alpha=PI_ALPHA)
        lo_log = ci.iloc[:,0].values
        hi_log = ci.iloc[:,1].values
        mean = clamp_nonneg(expm1_safe(mu_log, cap_original=cap))
        lo = clamp_nonneg(expm1_safe(lo_log, cap_original=cap))
        hi = clamp_nonneg(expm1_safe(hi_log, cap_original=cap))
        lo2 = np.minimum(lo, hi); hi2 = np.maximum(lo, hi)
        return (pd.Series(mean, index=idx), pd.Series(lo2, index=idx), pd.Series(hi2, index=idx))

    return best_model, f_mean, f_pi

def fit_ets_monthly_log_with_pi(ts_m: pd.Series):
    cap = compute_dynamic_cap(ts_m)
    y_log = np.log1p(ts_m)
    seasonal = 12 if len(ts_m) >= 24 else None
    model = ExponentialSmoothing(
        y_log, trend='add',
        seasonal=('add' if seasonal else None),
        seasonal_periods=seasonal
    ).fit()
    resid = np.array(y_log.values, dtype=float) - np.array(model.fittedvalues, dtype=float)
    sigma = float(np.nanstd(resid)) if np.isfinite(np.nanstd(resid)) else 0.5

    def f_mean(h_months: int):
        vals_log = np.array(model.forecast(h_months), dtype=float)
        idx = pd.period_range(ts_m.index[-1] + 1, periods=h_months, freq='M')
        mean = expm1_safe(vals_log, cap_original=cap)
        return pd.Series(clamp_nonneg(mean), index=idx)

    def f_pi(h_months: int):
        vals_log = np.array(model.forecast(h_months), dtype=float)
        idx = pd.period_range(ts_m.index[-1] + 1, periods=h_months, freq='M')
        std_log = np.full(h_months, sigma)
        mean, lo, hi = compute_pi_from_log_mean_std(vals_log, std_log, cap=cap, alpha=PI_ALPHA)
        return (pd.Series(mean, index=idx), pd.Series(lo, index=idx), pd.Series(hi, index=idx))

    return model, f_mean, f_pi


## 7) Blending (mean + PI)

We use inverse-error weights from rolling CV (sMAPE).  
For PI: combine log-scale variances via \(\sum w^2\sigma^2\).


In [18]:
def select_or_blend_forecasts(fc_dict: Dict[str, pd.Series],
                              cv_scores: Dict[str, float],
                              blend: bool = True):
    if not fc_dict:
        raise ValueError("No forecast candidates provided.")
    scores = {k: float(v) for k, v in (cv_scores or {}).items()
              if v is not None and np.isfinite(v)}
    if not blend:
        winner = next(iter(fc_dict.keys()))
        return fc_dict[winner], {'winner': winner, 'weights': {winner: 1.0}}
    if len(scores) == 0:
        keys = list(fc_dict.keys())
        idx = None
        for s in fc_dict.values():
            idx = s.index if idx is None else idx.union(s.index)
        w = {k: 1.0 / len(keys) for k in keys}
        blended = sum(w[k] * fc_dict[k].reindex(idx).fillna(0) for k in keys)
        return blended, {'winner': keys[0], 'weights': w}
    inv = {k: (1.0 / v if v > 0 else 0.0) for k, v in scores.items()}
    total = sum(inv.values())
    if total == 0:
        winner = min(scores, key=scores.get)
        return fc_dict[winner], {'winner': winner, 'weights': {winner: 1.0}}
    w = {k: inv[k] / total for k in inv}
    idx = None
    for s in fc_dict.values():
        idx = s.index if idx is None else idx.union(s.index)
    blended = sum(w.get(k, 0.0) * fc_dict[k].reindex(idx).fillna(0) for k in fc_dict)
    winner = min(scores, key=scores.get)
    return blended, {'winner': winner, 'weights': w}

def rolling_cv_monthly_adaptive(ts_m: pd.Series,
                                exo_monthly: Optional[pd.DataFrame] = None) -> Optional[Dict[str, float]]:
    n = len(ts_m)
    if n < 9:
        return None
    h = 3 if n >= 15 else 1
    min_train = max(12, n - (h + 2))
    splits = []
    for start in range(min_train, n - h + 1):
        train = ts_m.iloc[:start]
        test = ts_m.iloc[start:start + h]
        ex_train = exo_monthly.iloc[:start] if exo_monthly is not None else None

        metrics = {}
        mp, fp_mean, _ = fit_prophet_monthly_log_with_pi(train, ex_train)
        if fp_mean is not None:
            try:
                pv = np.array(fp_mean(h).values[:h], dtype=float)
                metrics['Prophet'] = smape(test.values, pv)
            except Exception:
                metrics['Prophet'] = 200.0
        try:
            _, fa_mean, _ = fit_arima_monthly_log_with_pi(train, ex_train)
            pv = np.array(fa_mean(h).values[:h], dtype=float)
            metrics['ARIMA'] = smape(test.values, pv)
        except Exception:
            metrics['ARIMA'] = 200.0
        try:
            _, fe_mean, _ = fit_ets_monthly_log_with_pi(train)
            pv = np.array(fe_mean(h).values[:h], dtype=float)
            metrics['ETS'] = smape(test.values, pv)
        except Exception:
            metrics['ETS'] = 200.0

        splits.append(metrics)
    dfm = pd.DataFrame(splits)
    return dfm.mean().to_dict()

def select_or_blend_forecasts_with_pi(
    fc_mean: Dict[str, pd.Series],
    fc_pi: Dict[str, Tuple[pd.Series, pd.Series, pd.Series]],
    cv_scores: Dict[str, float],
    blend: bool = True
):
    blended_mean, meta = select_or_blend_forecasts(fc_mean, cv_scores=cv_scores, blend=blend)

    if not fc_pi:
        idx = blended_mean.index
        return blended_mean, pd.Series(np.nan, index=idx), pd.Series(np.nan, index=idx), meta

    z = _z_from_alpha(PI_ALPHA)

    idx = None
    for s in fc_mean.values():
        idx = s.index if idx is None else idx.union(s.index)

    var_log = np.zeros(len(idx), dtype=float)
    weights = meta.get("weights") or {}
    for k, w in weights.items():
        if k not in fc_pi:
            continue
        mu_s, lo_s, hi_s = fc_pi[k]
        mu = mu_s.reindex(idx).astype(float).values
        lo = lo_s.reindex(idx).astype(float).values
        hi = hi_s.reindex(idx).astype(float).values

        mu_log = np.log1p(np.clip(mu, 0, None))
        lo_log = np.log1p(np.clip(lo, 0, None))
        hi_log = np.log1p(np.clip(hi, 0, None))

        sigma = (hi_log - lo_log) / (2.0 * z)
        sigma = np.clip(sigma, 1e-8, None)
        var_log += (float(w) ** 2) * (sigma ** 2)

    std_log = np.sqrt(np.clip(var_log, 1e-10, None))

    mu_bl = blended_mean.reindex(idx).astype(float).values
    mu_log_bl = np.log1p(np.clip(mu_bl, 0, None))

    cap = float(np.nanmax(mu_bl) * 6.0) if np.isfinite(np.nanmax(mu_bl)) and np.nanmax(mu_bl) > 0 else np.inf
    mean, lo, hi = compute_pi_from_log_mean_std(mu_log_bl, std_log, cap=cap, alpha=PI_ALPHA)

    return (pd.Series(mean, index=idx),
            pd.Series(lo, index=idx),
            pd.Series(hi, index=idx),
            meta)


## 8) Monthly forecast per Department + Language (with P05/P95)

In [19]:
def winsorize_monthly(ts_m: pd.Series, lower_q: float = 0.01, upper_q: float = 0.99) -> pd.Series:
    if ts_m.empty:
        return ts_m
    lo = ts_m.quantile(lower_q)
    hi = ts_m.quantile(upper_q)
    return ts_m.clip(lower=lo, upper=hi)

def forecast_per_dept_lang_monthly(incoming_clean: pd.DataFrame, exo_monthly: pd.DataFrame) -> pd.DataFrame:
    out_rows = []
    for (dept, lang), g_daily in incoming_clean.groupby(['department_id', 'language']):
        g_daily = g_daily.sort_values('Date')
        g_daily_clean = clean_outliers_daily(g_daily, method="IQR")

        g_m = (g_daily_clean.assign(month=g_daily_clean['Date'].dt.to_period('M'))
               .groupby('month')['ticket_total'].sum())
        g_m.index = pd.PeriodIndex(g_m.index, freq='M')
        if len(g_m) == 0:
            continue

        ts = winsorize_monthly(g_m, 0.01, 0.99)
        ex_m = prepare_monthly_exog(exo_monthly, ts.index)

        fc_mean, fc_pi = {}, {}
        cv = {}

        mp, fp_mean, fp_pi = fit_prophet_monthly_log_with_pi(ts, ex_m)
        if fp_mean is not None:
            try:
                fc_mean["Prophet"] = fp_mean(H_MONTHS)
                if ENABLE_MONTHLY_PI and fp_pi is not None:
                    fc_pi["Prophet"] = fp_pi(H_MONTHS)
            except Exception:
                pass

        try:
            _, fa_mean, fa_pi = fit_arima_monthly_log_with_pi(ts, ex_m)
            fc_mean["ARIMA"] = fa_mean(H_MONTHS)
            if ENABLE_MONTHLY_PI and fa_pi is not None:
                fc_pi["ARIMA"] = fa_pi(H_MONTHS)
        except Exception:
            pass

        try:
            _, fe_mean, fe_pi = fit_ets_monthly_log_with_pi(ts)
            fc_mean["ETS"] = fe_mean(H_MONTHS)
            if ENABLE_MONTHLY_PI and fe_pi is not None:
                fc_pi["ETS"] = fe_pi(H_MONTHS)
        except Exception:
            pass

        if not fc_mean:
            idx = pd.period_range(ts.index[-1] + 1, periods=H_MONTHS, freq='M')
            val = max(0.0, float(ts.mean()))
            fc_mean["NaiveMean"] = pd.Series([val]*H_MONTHS, index=idx)

        try:
            cv = rolling_cv_monthly_adaptive(ts, ex_m) or {}
        except Exception:
            cv = {}

        if ENABLE_MONTHLY_PI and len(fc_pi) >= 1:
            mean_s, p05, p95, meta = select_or_blend_forecasts_with_pi(fc_mean, fc_pi, cv_scores=cv, blend=True)
        else:
            mean_s, meta = select_or_blend_forecasts(fc_mean, cv_scores=cv, blend=True)
            p05 = pd.Series(np.nan, index=mean_s.index)
            p95 = pd.Series(np.nan, index=mean_s.index)

        for per in mean_s.index:
            out_rows.append({
                "department_id": str(dept),
                "language": str(lang),
                "month": per,
                "forecast_monthly": float(mean_s.loc[per]),
                "forecast_p05": float(p05.loc[per]) if per in p05.index else np.nan,
                "forecast_p95": float(p95.loc[per]) if per in p95.index else np.nan,
                "winner_model": meta.get("winner"),
            })

    df_out = pd.DataFrame(out_rows)
    if not df_out.empty:
        df_out["month"] = pd.PeriodIndex(df_out["month"], freq="M")
    return df_out


## 9) Department Accuracy (Backtesting)

In [20]:
def build_accuracy_by_dept_monthly(incoming: pd.DataFrame,
                                   exo_monthly: pd.DataFrame,
                                   mapping: pd.DataFrame) -> pd.DataFrame:
    d = apply_mapping(incoming.copy(), mapping)
    d["Date"] = pd.to_datetime(d["Date"])
    d["ticket_total"] = pd.to_numeric(d["ticket_total"], errors="coerce").fillna(0.0)
    d["month"] = d["Date"].dt.to_period("M")

    out = []
    for (dept, lang), g in d.groupby(["department_id", "language"]):
        gm = g.groupby("month")["ticket_total"].sum().sort_index()
        gm.index = pd.PeriodIndex(gm.index, freq="M")

        if len(gm) < (ACCURACY_MIN_TRAIN_MONTHS + ACCURACY_BACKTEST_MONTHS):
            continue

        eval_months = gm.index[-ACCURACY_BACKTEST_MONTHS:]
        preds, lo_s, hi_s, acts = [], [], [], []

        splits_done = 0
        for m in eval_months[::-1]:
            train_end = m - 1
            train = gm.loc[:train_end]
            if len(train) < ACCURACY_MIN_TRAIN_MONTHS:
                continue

            ex_train = prepare_monthly_exog(exo_monthly, train.index)

            fc_mean, fc_pi = {}, {}
            cv = {}

            mp, fp_mean, fp_pi = fit_prophet_monthly_log_with_pi(train, ex_train)
            if fp_mean is not None:
                try:
                    fc_mean["Prophet"] = fp_mean(ACCURACY_HORIZON_MONTHS)
                    if ENABLE_MONTHLY_PI and fp_pi is not None:
                        fc_pi["Prophet"] = fp_pi(ACCURACY_HORIZON_MONTHS)
                except Exception:
                    pass

            try:
                _, fa_mean, fa_pi = fit_arima_monthly_log_with_pi(train, ex_train)
                fc_mean["ARIMA"] = fa_mean(ACCURACY_HORIZON_MONTHS)
                if ENABLE_MONTHLY_PI and fa_pi is not None:
                    fc_pi["ARIMA"] = fa_pi(ACCURACY_HORIZON_MONTHS)
            except Exception:
                pass

            try:
                _, fe_mean, fe_pi = fit_ets_monthly_log_with_pi(train)
                fc_mean["ETS"] = fe_mean(ACCURACY_HORIZON_MONTHS)
                if ENABLE_MONTHLY_PI and fe_pi is not None:
                    fc_pi["ETS"] = fe_pi(ACCURACY_HORIZON_MONTHS)
            except Exception:
                pass

            if not fc_mean:
                continue

            try:
                cv = rolling_cv_monthly_adaptive(train, ex_train) or {}
            except Exception:
                cv = {}

            if ENABLE_MONTHLY_PI and len(fc_pi) >= 1:
                mean_s, p05, p95, _ = select_or_blend_forecasts_with_pi(fc_mean, fc_pi, cv_scores=cv, blend=True)
            else:
                mean_s, _ = select_or_blend_forecasts(fc_mean, cv_scores=cv, blend=True)
                p05 = pd.Series(np.nan, index=mean_s.index)
                p95 = pd.Series(np.nan, index=mean_s.index)

            if m in mean_s.index:
                preds.append(float(mean_s.loc[m]))
                lo_s.append(float(p05.loc[m]) if m in p05.index else np.nan)
                hi_s.append(float(p95.loc[m]) if m in p95.index else np.nan)
                acts.append(float(gm.loc[m]))
                splits_done += 1

            if splits_done >= ACCURACY_MAX_SPLITS:
                break

        if len(acts) == 0:
            continue

        y_true = np.array(acts, dtype=float)
        y_pred = np.array(preds, dtype=float)

        sm = smape(y_true, y_pred)
        mae = float(np.nanmean(np.abs(y_pred - y_true)))
        bias = float(np.nanmean((y_pred - y_true) / np.where(y_true > 0, y_true, np.nan)) * 100.0)
        acc = float(np.nanmean((1 - (np.abs(y_pred - y_true) / np.where(y_true > 0, y_true, np.nan))) * 100.0))

        cov = coverage_95(y_true, np.array(lo_s, dtype=float), np.array(hi_s, dtype=float)) if ENABLE_MONTHLY_PI else np.nan

        out.append({
            "department_id": str(dept),
            "language": str(lang),
            "vertical": g["vertical"].iloc[0] if "vertical" in g.columns else None,
            "department_name": g["department_name"].iloc[0] if "department_name" in g.columns else None,
            "Eval_Months": int(len(y_true)),
            "sMAPE_%": float(sm),
            "MAE": float(mae),
            "Bias_%": float(bias),
            "Accuracy_%": float(acc),
            "PI_Coverage_95_%": float(cov),
        })

    df = pd.DataFrame(out)
    if df.empty:
        return df
    return df.sort_values(["vertical", "department_id", "language"])


## 10) Staffing: Automatic safety factor (inflate/deflate) based on Accuracy

In [21]:
def compute_pi_width_ratio(fc_monthly: pd.DataFrame) -> pd.DataFrame:
    # PI width ratio proxy: (P95-P05)/Forecast.
    df = fc_monthly.copy()
    if not {"forecast_p05","forecast_p95","forecast_monthly"}.issubset(df.columns):
        df["pi_width_ratio"] = np.nan
        return df
    denom = df["forecast_monthly"].replace(0, np.nan).astype(float)
    df["pi_width_ratio"] = ((df["forecast_p95"] - df["forecast_p05"]) / denom).astype(float)
    return df

def staffing_safety_factor(accuracy_pct: float, pi_width_ratio: float, policy: dict) -> float:
    # Returns safety factor as decimal (e.g., +0.12 means +12%).
    if accuracy_pct is None or not np.isfinite(accuracy_pct):
        return policy.get("mid_accuracy_buffer_pct", 6.0) / 100.0

    low_thr = policy["low_accuracy_threshold"]
    mid_thr = policy["mid_accuracy_threshold"]
    high_thr = policy["high_accuracy_threshold"]

    if accuracy_pct < low_thr:
        return policy["low_accuracy_buffer_pct"] / 100.0

    if accuracy_pct < mid_thr:
        return policy["mid_accuracy_buffer_pct"] / 100.0

    if accuracy_pct >= high_thr:
        max_w = policy.get("max_pi_width_ratio_for_deflate", 0.35)
        if (pi_width_ratio is not None) and np.isfinite(pi_width_ratio) and (pi_width_ratio <= max_w):
            return policy["high_accuracy_deflate_pct"] / 100.0

    return 0.0

def apply_staffing_adjustment(daily_plan: pd.DataFrame,
                              acc_table: pd.DataFrame,
                              fc_monthly: pd.DataFrame,
                              policy: dict) -> pd.DataFrame:
    # Join daily staffing plan with dept accuracy and monthly PI width proxy, then adjust FTE.
    df = daily_plan.copy()
    df["department_id"] = df["department_id"].astype(str)
    df["language"] = df["language"].astype(str)

    if (acc_table is None) or acc_table.empty:
        acc = pd.DataFrame(columns=["department_id","language","Accuracy_%"])
    else:
        acc = acc_table[["department_id","language","Accuracy_%"]].copy()
    acc["department_id"] = acc["department_id"].astype(str)
    acc["language"] = acc["language"].astype(str)

    fm = compute_pi_width_ratio(fc_monthly)
    if "month" in fm.columns and not pd.api.types.is_period_dtype(fm["month"]):
        fm["month"] = pd.PeriodIndex(fm["month"], freq="M")
    fm = fm[["department_id","language","month","pi_width_ratio"]].copy()
    fm["department_id"] = fm["department_id"].astype(str)
    fm["language"] = fm["language"].astype(str)

    df["Date"] = pd.to_datetime(df["Date"])
    df["month"] = df["Date"].dt.to_period("M")

    df = df.merge(acc, on=["department_id","language"], how="left")
    df = df.merge(fm, on=["department_id","language","month"], how="left")

    sf = []
    for _, r in df.iterrows():
        sf.append(staffing_safety_factor(r.get("Accuracy_%"), r.get("pi_width_ratio"), policy))
    sf = np.array(sf, dtype=float)

    df["Safety_Factor_%"] = sf * 100.0
    base = pd.to_numeric(df["Capacity_FTE_per_day"], errors="coerce").fillna(0.0)
    df["FTE_Adj"] = np.clip(base * (1.0 + sf), 0.0, None)
    df["Buffer_FTE"] = df["FTE_Adj"] - base

    return df.drop(columns=["month"], errors="ignore")


## 11) Run pipeline end-to-end (forecast → accuracy → staffing with safety)

In [22]:
# 1) Load inputs
incoming = load_incoming(INCOMING_SOURCE_PATH, INCOMING_SHEET)
mapping = load_dept_map(DEPT_MAP_PATH, DEPT_MAP_SHEET)
incoming = apply_mapping(incoming, mapping)
prod_dept = load_productivity(PRODUCTIVITY_PATH)

# 2) Exogenous calendar
exo_daily, exo_monthly = build_exogenous_calendar(incoming, horizon_days=DAILY_HORIZON_DAYS)

# 3) Monthly forecast per dept/lang
fc_monthly = forecast_per_dept_lang_monthly(incoming, exo_monthly)
fc_monthly = fc_monthly.merge(mapping, on="department_id", how="left")
display(fc_monthly.head(10))

# 4) Accuracy by dept/lang (backtesting)
acc_dept = build_accuracy_by_dept_monthly(incoming, exo_monthly, mapping) if ENABLE_DEPT_ACCURACY_TABLE else pd.DataFrame()
display(acc_dept.head(10))

# 5) Create a daily plan from monthly forecast (simple allocation by business days)
def build_daily_plan_from_monthly(fc_m: pd.DataFrame, incoming_hist: pd.DataFrame, horizon_days: int) -> pd.DataFrame:
    last_date = incoming_hist["Date"].max()
    start = last_date + pd.Timedelta(days=1)
    end = start + pd.Timedelta(days=horizon_days - 1)

    rows = []
    for _, r in fc_m.iterrows():
        m = r["month"]
        if not isinstance(m, pd.Period):
            m = pd.Period(m, freq="M")

        month_days = pd.date_range(m.start_time, m.end_time, freq="D")
        month_days = month_days[(month_days >= start) & (month_days <= end)]
        if len(month_days) == 0:
            continue

        bdays = month_days[month_days.weekday < 5]
        denom = len(bdays) if len(bdays) > 0 else len(month_days)
        daily = float(r["forecast_monthly"]) / float(denom) if denom > 0 else 0.0

        for d in month_days:
            val = daily if (len(bdays) == 0 or d.weekday() < 5) else 0.0
            rows.append({
                "Date": d,
                "department_id": str(r["department_id"]),
                "language": str(r["language"]),
                "Forecast_Daily_Tickets": float(max(0.0, val)),
                "vertical": r.get("vertical"),
                "department_name": r.get("department_name"),
            })
    return pd.DataFrame(rows)

daily_plan = build_daily_plan_from_monthly(fc_monthly, incoming, DAILY_HORIZON_DAYS)

# Convert volume -> FTE using productivity
daily_plan = daily_plan.merge(prod_dept, on="department_id", how="left")
daily_plan["avg_tickets_per_agent_day"] = pd.to_numeric(daily_plan["avg_tickets_per_agent_day"], errors="coerce").fillna(0.0)
daily_plan["Capacity_FTE_per_day"] = np.where(
    daily_plan["avg_tickets_per_agent_day"] > 0,
    daily_plan["Forecast_Daily_Tickets"] / daily_plan["avg_tickets_per_agent_day"],
    np.nan
)

# 6) Apply safety factor policy
daily_plan_adj = apply_staffing_adjustment(daily_plan, acc_dept, fc_monthly, SAFETY_POLICY)
display(daily_plan_adj.head(10))

# 7) Export to Excel (Planet operational pack)
os.makedirs(OUTPUT_DIR, exist_ok=True)
with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl") as w:
    fc_monthly.sort_values(["vertical","department_id","language","month"]).to_excel(w, "forecast_monthly", index=False)
    daily_plan.to_excel(w, "daily_plan_base", index=False)
    daily_plan_adj.to_excel(w, "daily_capacity_plan", index=False)
    if acc_dept is not None and not acc_dept.empty:
        acc_dept.to_excel(w, "accuracy_dept_monthly", index=False)

print("✅ Export complete:", OUTPUT_XLSX)


13:27:35 - cmdstanpy - INFO - Chain [1] start processing
13:27:35 - cmdstanpy - INFO - Chain [1] done processing
13:27:36 - cmdstanpy - INFO - Chain [1] start processing
13:27:36 - cmdstanpy - INFO - Chain [1] done processing
13:27:37 - cmdstanpy - INFO - Chain [1] start processing
13:27:37 - cmdstanpy - INFO - Chain [1] done processing
13:27:38 - cmdstanpy - INFO - Chain [1] start processing
13:27:38 - cmdstanpy - INFO - Chain [1] done processing
13:27:39 - cmdstanpy - INFO - Chain [1] start processing
13:27:39 - cmdstanpy - INFO - Chain [1] done processing
13:27:39 - cmdstanpy - INFO - Chain [1] start processing
13:27:40 - cmdstanpy - INFO - Chain [1] done processing
13:27:40 - cmdstanpy - INFO - Chain [1] start processing
13:27:40 - cmdstanpy - INFO - Chain [1] done processing
13:27:41 - cmdstanpy - INFO - Chain [1] start processing
13:27:41 - cmdstanpy - INFO - Chain [1] done processing
13:27:42 - cmdstanpy - INFO - Chain [1] start processing
13:27:42 - cmdstanpy - INFO - Chain [1]

,department_id,language,month,forecast_monthly,forecast_p05,forecast_p95,winner_model,department_name,vertical
0,1,Chinese (Mandarin),2026-03,2.905117,2.252984,3.687986,ETS,CS_GT3C_EU,Payments
1,1,Chinese (Mandarin),2026-04,3.759032,2.964299,4.713087,ETS,CS_GT3C_EU,Payments
2,1,Chinese (Mandarin),2026-05,4.799669,3.831155,5.962342,ETS,CS_GT3C_EU,Payments
3,1,Chinese (Mandarin),2026-06,6.067856,4.887563,7.484767,ETS,CS_GT3C_EU,Payments
4,1,Chinese (Mandarin),2026-07,7.613353,6.174970,9.340093,ETS,CS_GT3C_EU,Payments
5,1,Chinese (Mandarin),2026-08,9.496797,7.743889,11.601115,ETS,CS_GT3C_EU,Payments
6,1,Chinese (Mandarin),2026-09,11.792085,10.081327,13.766955,ETS,CS_GT3C_EU,Payments
7,1,Chinese (Mandarin),2026-10,13.200000,12.579414,13.848947,ETS,CS_GT3C_EU,Payments
8,1,Chinese (Mandarin),2026-11,13.200000,13.199722,13.200278,ETS,CS_GT3C_EU,Payments
9,1,Chinese (Mandarin),2026-12,13.200000,13.199722,13.200278,ETS,CS_GT3C_EU,Payments


13:45:45 - cmdstanpy - INFO - Chain [1] start processing
13:45:45 - cmdstanpy - INFO - Chain [1] done processing
13:45:46 - cmdstanpy - INFO - Chain [1] start processing
13:45:46 - cmdstanpy - INFO - Chain [1] done processing
13:45:47 - cmdstanpy - INFO - Chain [1] start processing
13:45:47 - cmdstanpy - INFO - Chain [1] done processing
13:45:48 - cmdstanpy - INFO - Chain [1] start processing
13:45:48 - cmdstanpy - INFO - Chain [1] done processing
13:45:48 - cmdstanpy - INFO - Chain [1] start processing
13:45:49 - cmdstanpy - INFO - Chain [1] done processing
13:45:49 - cmdstanpy - INFO - Chain [1] start processing
13:45:51 - cmdstanpy - INFO - Chain [1] done processing
13:45:52 - cmdstanpy - INFO - Chain [1] start processing
13:45:52 - cmdstanpy - INFO - Chain [1] done processing
13:45:53 - cmdstanpy - INFO - Chain [1] start processing
13:45:53 - cmdstanpy - INFO - Chain [1] done processing
13:45:53 - cmdstanpy - INFO - Chain [1] start processing
13:45:54 - cmdstanpy - INFO - Chain [1]

,department_id,language,vertical,department_name,Eval_Months,sMAPE_%,MAE,Bias_%,Accuracy_%,PI_Coverage_95_%
7,10,English,Hospitality,CS_PMSP_PREM_L2,6,54.404584,133.628519,55.862855,13.642911,33.333333
8,10,French,Hospitality,CS_PMSP_PREM_L2,6,63.577449,21.648147,82.291163,-4.034759,66.666667
9,10,German,Hospitality,CS_PMSP_PREM_L2,6,97.678747,109.707287,203.525593,-166.618608,50.000000
10,10,Italian,Hospitality,CS_PMSP_PREM_L2,6,105.794356,53.798517,401.151147,-313.132679,33.333333
11,11,English,Hospitality,CS_PMSP_CLOUD_L2,6,74.225274,172.115649,47.181608,6.119581,16.666667
12,11,French,Hospitality,CS_PMSP_CLOUD_L2,6,116.883435,15.301505,467.561927,-442.802899,16.666667
13,11,German,Hospitality,CS_PMSP_CLOUD_L2,6,82.992847,105.924714,875.255158,-785.485271,66.666667
14,11,Italian,Hospitality,CS_PMSP_CLOUD_L2,6,132.077426,51.910037,854.452321,-754.452321,0.000000
41,23,English,Hospitality,CS_PMSP_FRANCE,6,49.151105,135.172169,231.816547,-156.353686,83.333333
42,23,French,Hospitality,CS_PMSP_FRANCE,6,110.974598,564.505954,405.396525,-356.526151,16.666667


,Date,department_id,language,Forecast_Daily_Tickets,vertical,department_name,avg_tickets_per_agent_day,Capacity_FTE_per_day,Accuracy_%,pi_width_ratio,Safety_Factor_%,FTE_Adj,Buffer_FTE
0,2026-03-01,1,Chinese (Mandarin),0.000000,Payments,CS_GT3C_EU,7.684139,0.000000,NaN,0.493957,6.0,0.000000,0.000000
1,2026-03-02,1,Chinese (Mandarin),0.132051,Payments,CS_GT3C_EU,7.684139,0.017185,NaN,0.493957,6.0,0.018216,0.001031
2,2026-03-03,1,Chinese (Mandarin),0.132051,Payments,CS_GT3C_EU,7.684139,0.017185,NaN,0.493957,6.0,0.018216,0.001031
3,2026-03-04,1,Chinese (Mandarin),0.132051,Payments,CS_GT3C_EU,7.684139,0.017185,NaN,0.493957,6.0,0.018216,0.001031
4,2026-03-05,1,Chinese (Mandarin),0.132051,Payments,CS_GT3C_EU,7.684139,0.017185,NaN,0.493957,6.0,0.018216,0.001031
5,2026-03-06,1,Chinese (Mandarin),0.132051,Payments,CS_GT3C_EU,7.684139,0.017185,NaN,0.493957,6.0,0.018216,0.001031
6,2026-03-07,1,Chinese (Mandarin),0.000000,Payments,CS_GT3C_EU,7.684139,0.000000,NaN,0.493957,6.0,0.000000,0.000000
7,2026-03-08,1,Chinese (Mandarin),0.000000,Payments,CS_GT3C_EU,7.684139,0.000000,NaN,0.493957,6.0,0.000000,0.000000
8,2026-03-09,1,Chinese (Mandarin),0.132051,Payments,CS_GT3C_EU,7.684139,0.017185,NaN,0.493957,6.0,0.018216,0.001031
9,2026-03-10,1,Chinese (Mandarin),0.132051,Payments,CS_GT3C_EU,7.684139,0.017185,NaN,0.493957,6.0,0.018216,0.001031


✅ Export complete: C:\Projects\Capacity_forecast_2026\outputs\capacity_forecast_hybrid.xlsx


## 12) Governance notes (Planet)

- Agree the **SAFETY_POLICY** with WFM + Ops and version it (change control).
- Monitor Accuracy% and Bias% by dept monthly; intervene if Bias > ±3%.
- When a department has Accuracy < 80% for 2+ cycles, prioritize a data quality / segmentation sprint.
